In [1]:
import tensorflow as tf
import numpy as np

In [6]:
# MNIST dataset parameters.
num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

# Training parameters.
learning_rate = 0.1
training_steps = 2000
batch_size = 256
display_step = 100

# Network parameters.
n_hidden_1 = 128 # 1st layer number of neurons.
n_hidden_2 = 256 # 2nd layer number of neurons.

In [17]:
# minist
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# Convert to float32.
x_train, y_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, y_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, y_test = x_train / 255., x_test / 255.

In [32]:
data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).repeat().shuffle(100).batch(batch_size)

In [33]:
# model
class NN(tf.keras.Model):
    def __init__(self, f1=64, f2=32, **kwargs):
        super(NN, self).__init__()
        
        self.fc1 = tf.keras.layers.Dense(f1)
        self.fc2 = tf.keras.layers.Dense(f2)
        self.out = tf.keras.layers.Dense(num_classes)
        
    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        x = tf.nn.softmax(x)
        return x

nn = NN()

In [19]:
for i,j in data.take(1):
    print(i)
    print(j)

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(256, 784), dtype=float32)
tf.Tensor(
[9 3 5 0 0 9 7 1 7 6 9 2 9 0 4 3 0 1 4 0 7 1 2 3 1 3 7 9 1 9 6 0 2 3 8 7 3
 5 4 3 1 7 4 0 4 0 4 4 0 1 3 6 8 0 7 4 1 1 3 0 9 1 1 8 4 5 4 6 7 0 6 8 1 9
 8 9 3 3 6 6 3 6 9 7 0 4 5 1 5 3 8 2 4 4 7 6 9 1 2 0 2 2 3 4 6 8 0 5 9 6 7
 5 2 3 7 7 1 0 3 0 7 7 3 6 0 8 5 8 7 6 9 7 3 5 5 0 2 7 7 0 2 1 0 2 8 7 1 0
 9 8 0 1 4 3 9 2 9 7 5 2 8 4 5 3 6 9 1 3 0 2 4 4 1 0 9 0 7 9 8 5 5 8 0 5 2
 4 6 6 8 2 4 6 5 5 8 1 5 3 5 2 4 1 0 3 6 1 3 2 6 5 8 7 6 4 8 9 3 9 1 4 0 8
 2 5 4 7 5 4 2 3 4 1 2 7 1 6 0 1 9 3 7 1 9 1 7 3 4 4 4 3 9 5 1 2 1 9], shape=(256,), dtype=uint8)


In [41]:
# loss
def cross_entropy_loss(logits, y):
#     y = tf.one_hot(y, num_classes, dtype=tf.int64)
    y = tf.cast(y, tf.int64)
    print(logits.shape)
    print(y.shape)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits), axis=-1)
    return loss

# metrics
def acc(y_pred, y):
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_pred, axis=-1), tf.cast(y, tf.int64)), tf.float32), axis=-1)
    return acc

#optimizer
optimizer = tf.optimizers.SGD(0.01)

In [42]:
# train_step
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = nn(x)
        loss = cross_entropy_loss(logits, y)
    
    grads = tape.gradient(loss, nn.trainable_weights)
    optimizer.apply_gradients(zip(grads, nn.trainable_weights))
    
    return loss

In [50]:
# train
for i, (x_train, y_train) in enumerate(data):
    loss = train_step(x_train, y_train)
    print('loss', loss.numpy())
    print(tf.argmax(nn(x_train), axis=-1))

(256, 10)
(256,)
loss 1.6665365
tf.Tensor(
[4 1 4 3 3 2 2 0 1 6 4 2 2 9 9 4 6 1 6 6 6 9 6 3 1 2 1 1 4 6 3 9 4 7 1 1 0
 4 6 7 3 0 2 3 6 2 0 3 9 0 9 0 4 4 4 9 0 1 7 3 0 7 7 6 2 7 3 3 2 6 3 6 6 7
 3 4 7 4 6 0 2 2 4 3 7 9 9 6 3 0 4 4 3 3 4 4 7 2 2 2 9 7 2 9 7 0 0 7 4 4 4
 1 2 1 1 2 9 9 9 4 7 9 9 0 3 1 0 6 1 0 0 3 9 3 3 0 6 0 2 0 7 3 0 1 3 9 1 3
 0 1 4 2 3 3 0 4 2 2 1 3 1 7 9 0 1 1 0 3 3 2 0 1 9 9 3 0 2 2 7 1 1 3 3 6 1
 6 3 2 3 9 2 7 4 4 7 6 4 6 9 6 6 1 3 9 4 7 7 2 1 3 4 4 4 3 1 3 1 6 2 1 7 3
 2 7 4 3 3 3 2 3 2 2 4 6 7 9 9 1 0 1 7 4 4 1 6 2 1 7 6 1 4 2 3 2 3 2], shape=(256,), dtype=int64)
(256, 10)
(256,)
loss 1.6648556
tf.Tensor(
[0 1 3 6 1 9 2 7 1 2 1 9 9 6 7 3 9 6 0 1 2 6 2 9 0 1 2 3 6 9 4 3 3 7 0 0 4
 9 6 4 3 9 0 4 0 1 0 2 6 3 6 9 0 7 7 9 1 1 9 7 9 7 0 1 2 4 6 7 1 9 0 3 6 0
 6 6 9 3 6 2 2 9 6 3 3 7 9 1 1 4 3 2 9 2 1 6 0 7 6 1 3 3 1 7 3 3 0 3 6 1 6
 2 7 9 7 2 6 3 3 0 2 0 3 4 2 6 9 3 7 1 6 0 4 9 4 4 2 6 4 1 0 1 7 4 2 2 4 3
 7 2 9 9 1 3 4 6 4 3 2 2 0 7 4 1 7 2 7 3 1 9 1 0 2 3 2 3 0 0 4 1 2

loss 1.661877
tf.Tensor(
[4 7 6 9 6 0 4 6 9 9 4 4 9 1 1 0 4 2 1 7 3 7 4 7 0 4 7 0 3 0 2 9 2 0 6 1 7
 9 2 1 7 4 4 0 9 9 7 6 9 7 3 3 0 9 9 2 4 7 4 0 9 0 3 3 4 4 2 2 4 9 7 1 2 4
 1 9 1 9 7 4 0 3 3 0 3 6 7 0 6 2 1 7 2 4 9 6 7 1 2 0 1 4 6 7 7 2 9 4 9 1 3
 7 2 4 0 6 2 6 0 2 3 7 4 9 7 4 3 6 3 2 9 0 9 3 6 7 7 6 0 3 3 1 0 6 2 2 3 1
 6 2 6 0 0 6 9 9 3 4 2 3 6 2 9 1 6 9 2 1 1 2 9 4 3 2 1 3 0 9 2 0 7 7 6 7 3
 1 6 4 3 3 1 1 4 0 9 7 7 1 4 9 6 0 2 3 3 0 0 3 1 1 1 4 9 1 0 1 3 6 1 3 7 1
 7 9 9 7 3 3 3 2 7 4 0 6 4 2 4 4 3 7 2 3 2 7 7 9 0 7 3 1 3 4 9 4 0 1], shape=(256,), dtype=int64)
(256, 10)
(256,)
loss 1.6654469
tf.Tensor(
[7 3 2 1 3 0 7 6 9 4 1 1 1 2 3 2 2 9 3 1 2 6 9 6 0 4 1 3 4 6 4 3 1 2 2 6 0
 7 0 7 4 4 3 9 9 3 1 6 3 7 4 3 6 1 6 2 9 0 4 2 0 9 3 7 4 7 4 2 9 6 3 4 1 2
 6 3 3 9 3 4 6 3 3 0 0 0 0 3 1 1 2 0 7 2 9 9 1 2 2 2 3 3 2 4 1 3 0 6 7 9 3
 1 2 0 1 6 1 2 6 1 0 0 2 0 2 7 7 6 1 3 2 9 0 0 3 2 7 3 9 6 4 9 7 2 4 9 0 3
 2 1 0 3 4 9 3 1 9 1 6 7 4 3 9 1 3 2 3 4 7 3 1 6 9 0 1 1 7 0 4 1 0 6 4 3 2
 4 4 0 3 

loss 1.6621395
tf.Tensor(
[3 1 4 2 0 0 3 1 9 7 2 7 3 2 1 6 6 1 0 3 2 1 7 4 4 9 1 7 4 4 1 0 7 1 9 4 2
 3 2 2 0 3 2 3 9 7 4 3 7 6 0 2 6 9 0 9 0 1 0 4 3 0 2 0 2 4 4 2 1 1 6 7 6 9
 0 9 7 7 2 4 6 2 3 0 9 1 2 1 3 3 1 0 3 6 3 2 0 4 9 7 1 0 1 2 7 9 6 4 3 1 0
 3 2 9 0 3 6 2 3 4 6 3 7 3 0 7 6 4 0 0 6 4 1 2 2 3 2 0 9 2 7 1 9 7 9 4 0 0
 1 0 1 1 9 7 4 3 1 1 4 2 9 6 3 2 3 2 2 3 4 6 3 9 3 6 3 4 7 9 7 0 6 3 7 4 9
 7 1 0 2 4 1 1 3 3 7 3 1 2 6 0 4 4 7 7 3 7 3 7 3 3 6 7 3 2 3 1 3 7 7 0 0 7
 2 1 4 9 9 7 2 2 9 3 2 3 2 1 3 4 9 0 0 6 1 4 2 4 3 0 2 9 1 3 6 6 1 6], shape=(256,), dtype=int64)
(256, 10)
(256,)
loss 1.6736999
tf.Tensor(
[6 6 0 1 6 0 6 0 7 9 1 3 7 4 9 6 3 9 3 6 9 4 4 9 4 6 2 3 4 4 3 2 4 7 9 4 1
 4 0 2 9 2 4 9 2 1 9 9 7 0 7 7 7 9 9 2 6 1 9 9 4 1 4 0 1 6 6 3 2 1 3 4 4 0
 0 0 2 0 7 2 7 9 4 0 3 3 3 7 9 3 4 4 6 7 2 6 1 1 2 7 6 3 2 7 9 3 4 1 4 9 2
 4 6 0 1 4 0 9 2 1 1 0 4 1 4 0 3 9 0 6 1 3 7 1 7 6 2 3 3 3 0 3 4 7 2 3 6 2
 0 2 2 6 1 2 7 3 6 3 4 2 1 4 1 6 2 7 4 2 0 4 6 9 0 0 3 6 1 6 1 7 2 0 7 0 3
 1 2 3 4

loss 1.6678
tf.Tensor(
[6 1 6 6 4 9 0 4 9 3 3 3 3 9 9 0 0 9 7 6 4 0 9 6 4 0 2 4 7 3 0 3 3 9 1 7 3
 6 9 9 9 4 3 9 1 2 2 4 1 9 3 4 4 7 9 1 2 0 9 1 2 4 2 7 3 7 6 3 9 3 1 2 2 2
 1 2 6 2 2 2 2 7 3 9 6 3 1 6 3 4 2 6 1 9 6 0 3 4 7 9 4 2 4 7 2 1 9 7 3 0 4
 0 0 3 3 2 3 3 2 3 7 0 0 3 3 6 3 4 7 4 6 6 1 0 7 0 0 3 0 0 4 3 1 1 6 1 0 6
 2 0 2 6 0 4 4 4 3 1 7 9 0 6 2 9 2 3 7 1 7 9 1 4 3 7 2 0 3 1 0 4 2 3 3 4 7
 7 1 2 2 9 4 1 9 1 4 7 4 1 1 3 7 3 9 1 3 3 6 7 6 1 4 0 1 4 7 9 6 1 1 1 3 0
 6 6 9 1 1 1 2 9 7 2 3 0 3 0 9 6 1 1 0 6 6 7 3 6 3 9 4 7 3 2 3 9 0 7], shape=(256,), dtype=int64)
(256, 10)
(256,)
loss 1.6594
tf.Tensor(
[0 4 3 4 0 9 4 6 0 9 0 2 6 0 2 7 7 6 1 7 9 3 6 4 3 7 0 0 2 0 2 2 9 3 7 2 3
 7 4 2 7 4 9 1 0 9 2 3 9 9 7 6 9 4 1 0 4 4 1 6 3 4 7 9 6 1 2 3 7 6 9 2 9 6
 3 1 3 9 2 2 6 9 6 4 2 1 6 3 9 9 1 4 4 1 1 9 0 4 2 0 1 0 6 2 2 3 4 7 0 9 9
 2 6 9 3 4 2 0 4 2 7 1 7 7 2 3 2 7 3 7 1 7 2 1 1 0 3 1 3 4 3 6 3 1 0 3 6 6
 7 4 6 0 3 2 2 2 7 1 3 3 0 9 0 2 4 0 0 3 7 1 9 4 0 0 6 0 4 1 9 9 1 7 9 7 0
 7 9 3 2 2 7 1

(256,)
loss 1.6974678
tf.Tensor(
[3 2 1 0 2 0 1 0 4 4 1 1 2 7 7 3 2 1 9 6 9 6 2 0 9 6 3 3 0 6 7 7 2 0 2 0 1
 7 9 7 6 9 1 6 6 4 2 7 3 2 3 2 3 9 2 7 4 0 1 3 9 1 1 7 0 6 3 9 4 3 9 3 2 3
 1 6 3 6 0 3 3 4 7 0 7 4 2 6 2 0 9 0 3 0 9 3 0 4 3 3 1 6 9 1 1 6 3 9 4 7 9
 1 3 4 2 7 4 0 0 6 4 3 4 2 9 6 7 4 2 1 1 1 0 3 7 1 4 3 2 4 6 3 1 2 9 9 6 0
 7 0 4 2 6 9 0 0 1 2 1 3 1 3 7 1 4 3 3 6 0 2 4 4 6 4 7 4 7 2 3 7 4 4 0 6 2
 0 3 2 4 1 9 0 7 4 7 3 0 6 7 3 3 1 0 3 7 1 2 9 2 1 2 6 4 4 4 4 9 3 6 3 2 1
 7 6 1 9 1 7 3 6 3 3 2 9 7 9 7 3 4 6 0 9 4 2 3 1 1 3 1 7 2 0 4 3 3 0], shape=(256,), dtype=int64)
(256, 10)
(256,)
loss 1.6294267
tf.Tensor(
[7 0 9 1 2 4 9 1 4 3 0 0 4 7 0 9 4 0 0 6 2 6 6 4 3 0 6 7 1 6 6 2 1 2 0 0 7
 6 4 3 2 3 6 7 9 4 1 3 7 1 7 3 2 4 2 2 0 4 0 3 7 9 6 0 9 4 7 6 1 2 3 0 1 2
 0 3 2 3 0 6 3 0 2 3 1 3 6 7 2 2 9 1 9 3 0 1 6 2 9 2 2 6 3 0 2 1 3 6 7 9 3
 7 1 4 1 7 7 9 4 4 2 7 1 2 0 9 0 0 9 2 4 4 4 2 3 3 1 3 3 1 4 6 1 2 4 9 6 3
 4 3 3 0 9 1 6 6 9 1 9 2 9 2 7 4 7 0 2 0 1 0 3 7 3 2 7 4 9 3 1 0 6 1 9 1 7
 

...

loss 1.6326516
tf.Tensor(
[6 7 2 4 4 1 4 2 7 1 9 3 3 6 7 0 7 9 3 6 4 2 3 3 4 7 0 0 3 6 4 0 1 7 6 9 9
 0 6 6 3 0 0 2 2 4 2 4 4 2 0 4 9 2 2 9 4 3 2 1 3 6 6 6 7 3 0 7 0 2 9 0 0 6
 1 2 2 0 6 2 4 7 3 3 0 9 1 9 9 9 0 1 2 4 3 7 2 4 9 0 4 3 7 2 4 6 2 7 5 9 0
 9 7 1 9 1 0 1 2 6 7 9 3 4 1 9 1 9 6 3 7 7 6 6 3 7 7 9 7 3 3 3 9 6 1 2 0 5
 3 2 5 1 4 2 0 2 3 6 6 7 9 7 7 4 2 1 6 6 4 0 3 2 1 3 4 7 1 4 1 0 2 7 1 1 3
 9 7 6 9 1 2 0 2 6 4 3 7 9 1 7 4 0 7 4 7 3 2 3 5 3 0 1 6 0 1 9 3 2 4 4 0 9
 9 1 1 2 7 9 4 6 9 6 3 3 6 4 4 9 7 2 4 3 3 6 4 1 9 0 1 6 7 1 2 2 3 6], shape=(256,), dtype=int64)
(256, 10)
(256,)
loss 1.6277602
tf.Tensor(
[4 0 6 6 3 3 7 1 7 6 4 6 2 1 6 9 9 9 1 4 7 6 6 9 9 1 4 9 7 0 3 3 1 3 3 0 7
 9 6 9 2 3 4 0 0 9 2 4 3 0 6 4 7 1 1 4 4 2 2 3 1 3 3 2 2 9 7 1 1 0 4 0 1 0
 2 3 9 7 0 0 3 3 4 3 7 0 7 6 2 5 2 1 6 1 2 3 0 0 3 6 4 2 3 3 0 7 4 3 2 1 7
 1 0 3 0 3 1 6 9 7 0 4 9 3 2 9 6 4 2 7 9 9 4 3 1 4 3 3 4 1 7 0 1 3 0 2 4 0
 1 6 6 1 9 3 0 0 7 7 1 6 9 0 7 3 0 3 2 3 1 7 1 4 2 4 5 2 0 6 2 0 1 7 0 2 1
 6 4 3 7

loss 1.6168474
tf.Tensor(
[1 4 7 6 0 3 4 3 4 0 1 1 2 9 9 3 3 6 1 3 6 1 7 4 9 9 6 3 9 2 0 4 1 1 5 0 0
 6 2 7 3 7 7 7 4 0 0 1 1 6 9 2 2 2 6 0 0 3 6 7 2 2 7 2 4 6 3 2 2 2 7 2 6 3
 4 1 4 9 0 1 9 2 4 6 6 5 4 6 7 4 6 0 1 0 4 3 1 6 2 0 1 7 7 9 9 9 2 3 3 1 3
 2 9 3 3 2 1 4 1 1 6 3 3 3 1 4 9 4 0 6 0 6 3 9 9 1 3 4 1 6 6 3 0 9 1 1 7 2
 0 1 4 7 1 6 9 0 4 2 7 7 5 3 1 3 7 1 9 0 6 0 9 2 0 2 3 4 3 1 3 3 6 4 9 6 3
 7 2 3 1 5 7 0 9 1 9 4 4 1 7 3 9 4 2 6 3 5 0 4 9 1 4 0 0 3 3 3 2 6 4 0 4 2
 0 6 9 9 4 1 4 7 7 6 7 2 9 2 7 6 4 9 7 9 4 3 0 1 4 2 0 4 3 2 1 7 7 0], shape=(256,), dtype=int64)
(256, 10)
(256,)
loss 1.6412971
tf.Tensor(
[6 4 6 4 6 0 2 9 6 3 3 2 3 4 7 4 1 9 0 0 4 7 7 1 3 2 0 2 9 2 2 3 2 3 2 1 4
 6 6 3 9 2 0 0 9 2 0 7 3 1 0 4 0 2 1 1 3 6 6 3 0 1 7 7 1 7 0 3 4 3 4 6 6 1
 4 6 6 9 0 4 6 6 2 0 9 6 1 7 5 9 7 9 3 3 1 1 6 0 2 1 3 0 6 3 2 9 1 3 2 3 1
 2 4 9 7 7 6 6 0 3 3 2 9 3 0 2 3 1 2 7 9 7 4 9 7 3 0 1 2 0 9 0 6 3 7 7 3 3
 0 2 4 1 1 1 0 5 0 4 9 3 3 1 2 4 6 2 6 9 4 6 4 0 6 4 1 6 9 6 1 7 2 3 5 1 2
 3 0 7 4

(256, 10)
(256,)
loss 1.6250873
tf.Tensor(
[7 4 4 3 2 3 1 4 4 4 0 9 0 1 4 4 7 9 3 9 1 1 2 3 2 1 7 1 4 6 0 0 4 3 4 1 9
 3 2 9 6 9 7 2 2 4 0 2 3 6 0 1 1 1 3 7 2 1 9 2 0 5 0 2 2 9 9 2 1 1 4 6 2 6
 3 2 2 4 3 6 9 1 4 6 7 2 1 3 3 7 6 0 0 9 6 2 1 1 6 6 9 9 6 1 3 6 7 9 1 9 1
 1 3 3 2 3 1 0 2 9 9 3 4 1 3 3 7 4 5 0 7 9 4 0 3 7 3 9 3 4 3 2 7 0 0 0 3 0
 7 6 6 0 2 0 4 4 7 7 4 3 9 1 7 2 7 6 3 7 1 6 0 6 9 0 1 4 6 1 6 5 0 0 6 1 3
 2 9 1 4 0 9 7 0 9 2 6 0 5 7 0 3 4 7 4 3 6 0 3 4 3 7 6 6 1 1 3 1 4 3 2 7 2
 2 4 0 4 3 3 7 3 0 0 3 1 6 1 9 3 2 1 6 1 3 3 3 3 1 9 3 4 7 4 9 1 6 4], shape=(256,), dtype=int64)
(256, 10)
(256,)
loss 1.6440337
tf.Tensor(
[6 6 5 2 9 4 4 4 4 9 3 9 0 4 7 0 1 9 7 9 3 6 4 7 3 1 6 0 1 4 4 3 3 6 2 3 4
 6 2 6 6 7 3 7 6 2 9 7 1 6 3 4 2 0 9 1 3 0 1 3 6 2 1 6 0 2 6 3 7 6 2 0 1 0
 3 6 4 5 6 2 6 6 2 6 1 9 6 0 4 9 1 2 3 2 2 4 2 0 2 1 2 2 5 0 2 0 4 1 1 2 2
 0 9 0 7 2 3 9 1 7 1 7 6 0 0 7 9 0 2 1 3 1 4 4 0 9 9 4 6 7 7 3 6 3 4 4 0 6
 0 3 1 3 1 7 2 0 4 3 1 0 1 3 3 9 7 7 1 4 9 9 3 0 9 3 3 9 2 1 3 5 2

loss 1.6282718
tf.Tensor(
[4 0 3 2 9 7 0 7 1 0 1 3 6 7 4 1 7 9 2 5 2 0 3 6 3 4 9 0 7 6 0 1 4 6 6 3 2
 3 3 2 7 2 0 0 6 0 0 2 6 9 7 0 1 3 9 6 9 7 9 1 9 7 1 4 7 1 3 0 9 3 2 7 6 2
 9 0 9 0 3 2 3 2 9 1 9 2 0 5 4 3 2 6 6 3 6 1 7 7 1 6 9 1 3 4 0 0 2 3 9 2 0
 1 4 0 1 0 1 3 4 2 4 2 6 2 0 0 1 3 1 4 9 1 9 7 1 0 4 1 6 6 6 7 9 3 7 3 0 7
 4 9 3 0 1 3 4 0 9 4 6 9 3 1 1 3 2 2 1 7 1 1 6 7 9 3 3 2 6 7 7 3 2 4 0 9 3
 2 7 0 2 4 9 0 4 2 7 1 1 4 7 3 9 7 5 9 5 3 1 1 4 6 2 3 2 0 1 2 3 3 0 1 6 3
 1 3 7 6 6 4 4 7 2 7 9 6 1 1 3 6 9 0 3 4 7 2 0 0 3 2 2 4 6 1 3 1 4 3], shape=(256,), dtype=int64)
(256, 10)
(256,)
loss 1.6458678
tf.Tensor(
[9 6 9 2 1 9 2 4 4 3 1 6 1 3 7 6 3 1 4 6 7 3 3 2 5 1 4 2 7 4 0 6 0 9 6 4 6
 6 7 3 4 6 4 2 7 0 2 9 9 0 3 9 4 3 0 9 2 9 6 6 3 4 0 4 4 3 2 4 2 7 4 4 5 2
 0 1 0 4 0 3 4 7 3 2 0 1 0 9 4 6 3 9 6 3 2 4 6 2 9 2 6 9 0 3 2 3 9 9 1 6 1
 3 1 0 0 9 3 7 2 9 2 4 3 0 1 9 3 2 6 1 1 3 4 1 1 2 0 4 4 0 0 3 9 7 3 1 2 7
 2 4 1 7 6 1 3 2 0 4 6 6 6 7 6 0 5 3 7 7 0 2 6 3 9 4 2 7 4 3 4 9 4 2 4 4 6
 7 9 0 7

KeyboardInterrupt: 